# Exercise 2.2 - Random Matrix theory

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm as tqdm_

import disordered_sys.generate

In [2]:
epsilon = 1e-3
c = 3
n = 2**6
lam = 0

m, g = disordered_sys.generate.generate_m(c, n, return_g=True)

In [3]:
def get_neighbors(g, k, j=-1):
    neighbors_set = set(g.neighbors(k)) - {j}
    return np.array(list(neighbors_set))

def marginal_equation(m, g, marg_precs, lam, k):
    neighbor_idx = get_neighbors(g, k)
    neighbor_weight = m[neighbor_idx, k]

    return epsilon + 1j * lam + sum(neighbor_weight / marg_precs[neighbor_idx])

def cavity_equation(m, g, cav_precs, lam, k, j):
    neighbor_idx = get_neighbors(g, k, j)
    neighbor_weight = m[neighbor_idx, k]
    cav_idx = np.full_like(neighbor_idx, j)

    return epsilon + 1j * lam + sum(
        neighbor_weight / cav_precs[neighbor_idx, cav_idx]
    )

In [4]:
steps = n**3
# cavity_precs = np.array([[np.inf] * n] * n, dtype=np.complex64)
cavity_precs = np.ones((n, n)).astype(np.complex64)
walker = iter(disordered_sys.random_walk.random_walk(g, steps))

last = next(walker)
for node in tqdm_(walker, total=steps):
    cavity_precs[last,
                 node] = cavity_equation(m, g, cavity_precs, lam, node, last)
    last = node

walker = iter(disordered_sys.random_walk.random_walk(g, steps=steps))
for node in tqdm_(walker, total=steps):
    cavity_precs[node,
                 node] = cavity_equation(m, g, cavity_precs, lam, node, node)

  0%|          | 0/262144 [00:00<?, ?it/s]

  0%|          | 0/262144 [00:00<?, ?it/s]

In [ ]:
# plt.scatter(np.arange(n), np.real(cavity_precs))
# plt.ylim(-1e1, 1e1)
plt.matshow(cavity_precs.real)
plt.show()

plt.matshow(cavity_precs.imag)
plt.show()

plt.hist(cavity_precs.real)
plt.show()

plt.scatter(np.arange(n), np.real(np.diag(cavity_precs)))
plt.ylim(-1e1, 1e1)
plt.show()

In [ ]:
plt.hist(np.real(cavity_precs), range=(-10, 10), bins=100)